In [4]:
# bm25
import os
import numpy as np
import math
import pickle
import re
import pandas as pd
from tqdm import tqdm
import pickle


In [5]:
def process_text(text):
    text = str(text)
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    text = text.replace('\n',' ').replace('\t' , ' ')
    text = text.strip()
    return text.split()

In [6]:


    
doc_df = pd.read_csv('documents.csv')
docs_id = doc_df['doc_id'].tolist()
docs = doc_df['doc_text'].apply(process_text).tolist()
doc_id2idx = {}
for i in range(len(docs_id)):
    doc_id2idx[docs_id[i]] = i


In [7]:
train_query_df = pd.read_csv('train_queries.csv')
train_queries = train_query_df['query_text'].apply(process_text).tolist()
train_queries_id = train_query_df['query_id'].astype('str').tolist()
train_queries_pos_docs = train_query_df['pos_doc_ids'].tolist()
train_queries_top_bm25 = train_query_df['bm25_top1000'].tolist()
train_queries_top_bm25_scores = train_query_df['bm25_top1000_scores'].tolist()

print(len(train_queries))
print(len(train_queries_id))
print(len(train_queries_pos_docs))
print(len(train_queries_top_bm25))
print(len(train_queries_top_bm25_scores))

120
120
120
120
120


In [8]:
test_query_df = pd.read_csv('test_queries.csv')
test_queries = test_query_df['query_text'].apply(process_text).tolist()
test_queries_id = test_query_df['query_id'].astype('str').tolist()
test_queries_top_bm25 = test_query_df['bm25_top1000'].tolist()
test_queries_top_bm25_scores = test_query_df['bm25_top1000_scores'].tolist()

print(len(test_queries))
print(len(test_queries_id))
print(len(test_queries_top_bm25))
print(len(test_queries_top_bm25_scores))

80
80
80
80


In [9]:
# query_vocab = set()
# for query in train_queries:
#     query_vocab.update(query)
    
# for query in test_queries:
#     query_vocab.update(query)
# print(len(query_vocab))

445


In [10]:
# vocab = set()
# total_len = 0
# doc_tfs = []
# doc_lens = []
# for doc in docs:
#     doc_tf = {}
#     total_len+=len(doc)
#     doc_lens.append(len(doc))
#     for word in doc:
#         if word in query_vocab:
#             vocab.add(word)
#         if word in vocab:    
#             if word in doc_tf:
#                 doc_tf[word] += 1
#             else:
#                 doc_tf[word] = 1
#     doc_tfs.append(doc_tf)
# print(len(vocab))
# vocab = vocab | query_vocab
# print(len(vocab))
# avg_doc_len = total_len / len(docs)
# print(avg_doc_len)

442
445
520.59244


In [11]:
# # hyper params
# K1 = 0.8
# b = 0.7
# K3 = 1000
# delta  = 0.2

# # Discriminative power
# doc_idfs = {}
# for word in vocab:
#     # avoid division by zero
#     ni = 0
#     for doc_tf in doc_tfs:
#         if word in doc_tf:
#             ni += 1
#     idf = math.log((len(docs_id) - ni + 0.5) / (ni + 0.5))
#     doc_idfs[word] = idf


# # compute tfidf
# doc_tfidf = []
# idx = 0
# for doc_tf in doc_tfs:
#     tfidf = []
#     doc_len_normalize = doc_lens[idx] / avg_doc_len
#     idx += 1
#     for word in query_vocab:
#         # calculate tf
#         tf = 0
#         if word in doc_tf:
#             tf = doc_tf[word]
#         if tf > 0:
#             tf = tf / (1 - b + b * doc_len_normalize)
#             tf = ((K1 + 1) * (tf + delta)) / (K1 + tf + delta)
#         idf = doc_idfs[word]
#         tfidf.append(tf * idf * (idf))
#     doc_tfidf.append(tfidf)
# doc_tfidf = np.array(doc_tfidf)

# print(doc_tfidf.shape)


In [9]:
# def map_score(x,ans):
#     score = 0.0
#     results = x
#     relates = ans
#     idx = 0
#     count = 0
#     for result in results:
#         if result in relates:
#             count += 1
#             score += count / (idx + 1)
#         idx += 1
#     score /= len(relates)
#     return score
    

In [12]:
# BM25 = []
# query_idx = 0
# _map = 0.0
# for query_id, query in zip(train_queries_id, train_queries):
#     query_tf = [0] * len(vocab)
#     for q in query:
#         idx = 0
#         for word in vocab:
#             if word == q:
#                 query_tf[idx] += 1
#                 if query_tf[idx] > 1:
#                     print('repeat word in query')
#                 break
#             idx += 1
#     for i in range(len(query_vocab)):
#         query_tf[i] = ((K3+1) * query_tf[i]) / (K3+query_tf[i])
#     query_tf = np.array(query_tf)
#     relevance = np.matmul(doc_tfidf, query_tf)
#     query_bm25_doc_idx = np.argsort(relevance)[-1000:]
#     BM25.append(query_bm25_doc_idx)
# #     label
#     query_bm25_ans = train_queries_top_bm25[query_idx].split()
#     query_bm25 = []
#     for doc_idx in query_bm25_doc_idx:
#         query_bm25.append(docs_id[doc_idx])

#     _ap = map_score(query_bm25,query_bm25_ans)
#     _map += _ap
#     query_idx += 1

# print(_map / len(train_queries))

In [13]:
# use my self bm25

# x_qd = []
# train_y = []

# for query_idx,query in enumerate(train_queries):
#     query = ' '.join(query)
# #     BM25找回來的
#     bm25_topk = BM25[query_idx]
# #     query的正確答案
#     query_pos_doc_idx = []
#     query_pos_doc_id = train_queries_pos_docs[query_idx].split()
#     count = 0
#     for doc_id in query_pos_doc_id:
#         doc_idx = doc_id2idx[doc_id]
#         if doc_idx in bm25_topk:
#             count += 1
#         else:
#             document = ' '.join(docs[doc_idx])
#             x_qd.append((query,document))
#             train_y.append(1)
            
#         query_pos_doc_idx.append(doc_idx)
#     count = 0
#     for doc_idx in bm25_topk:
#         document = ' '.join(docs[doc_idx])
#         if doc_idx in query_pos_doc_idx:
#             x_qd.append((query,document))
#             train_y.append(1)
#             count += 1
#         else:
# #             if np.random.random()<0.1:
#             x_qd.append((query,document))
#             train_y.append(0)

# # x_d = np.array(x_d)
# # x_q = np.array(x_q)
# # y = np.array(y)
# print(len(x_qd))
# print(len(train_y))

# print(sum(train_y) / len(train_y))
# import torch
# train_y = torch.tensor(train_y)

    

In [14]:
x_qd = []
train_y = []

for query_idx,query in enumerate(train_queries):
    query = ' '.join(query)
#     BM25找回來的
    bm25_topk = train_queries_top_bm25[query_idx].split()
#     query的正確答案
    query_pos_doc_idx = []
    query_pos_doc_id = train_queries_pos_docs[query_idx].split()
    for doc_id in query_pos_doc_id:
        doc_idx = doc_id2idx[doc_id]  
        query_pos_doc_idx.append(doc_idx)
        
    for doc_id in bm25_topk:
        doc_idx = doc_id2idx[doc_id]
        document = ' '.join(docs[doc_idx])
        if doc_idx in query_pos_doc_idx:
            x_qd.append((query,document))
            train_y.append(1)
        else:
            x_qd.append((query,document))
            train_y.append(0)
        


print(len(x_qd))
print(len(train_y))

print(sum(train_y) / len(train_y))
import torch
train_y = torch.tensor(train_y)

    

120000
120000
0.047825


In [20]:
import torch
import transformers
from transformers import RobertaTokenizer, RobertaForQuestionAnswering , RobertaForSequenceClassification
from transformers import BertTokenizer, RobertaForQuestionAnswering , RobertaForSequenceClassification

# pretrained_model = 'roberta-large'
pretrained_model = 'bert-base-uncased'
# tokenizer = RobertaTokenizer.from_pretrained(pretrained_model)
tokenizer = BertTokenizer.from_pretrained(pretrained_model)

In [21]:
input_dict = {}
input_ids = []
attention_mask = []
token_type_ids = []

for i in tqdm(range(len(x_qd))):
    x = x_qd[i]
    tmp_dict =  tokenizer.encode_plus(x[0],
                          x[1],
                          max_length=512,
                          return_tensors='pt',
                          return_token_type_ids = True,
                          pad_to_max_length=True,
                          truncation=True)
    input_ids.append(tmp_dict['input_ids'][0])
    attention_mask.append(tmp_dict['attention_mask'][0])
    token_type_ids.append(tmp_dict['token_type_ids'][0])
    
#     print(input_ids[0])
#     print(attention_mask[0])
#     print(token_type_ids[0])
#     input()


print(len(input_ids))
print(len(attention_mask))
print(len(token_type_ids))


# with open('input_dict{}'.format(len(x_qd)), 'wb') as handle:
#     pickle.dump(input_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('train_y{}'.format(len(x_qd)), 'wb') as handle:
#     pickle.dump(train_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 120000/120000 [38:24<00:00, 52.06it/s]   

120000
120000
120000


In [22]:
with open('{}_input_ids{}'.format(pretrained_model,len(x_qd)), 'wb') as handle:
    pickle.dump(input_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('{}_attention_mask{}'.format(pretrained_model,len(x_qd)), 'wb') as handle:
    pickle.dump(attention_mask, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('{}_token_type_ids{}'.format(pretrained_model,len(x_qd)), 'wb') as handle:
    pickle.dump(token_type_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('train_y{}'.format(len(x_qd)), 'wb') as handle:
    pickle.dump(train_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
# input_dict = {}

# count = 0 
# for i in tqdm(range(len(x_qd))):
#     x = x_qd[i]
#     tmp_dict =  tokenizer(x[0],
#                           x[1],
#                           max_length=512,
#                           return_tensors='pt',
#                           return_token_type_ids = True,
#                           pad_to_max_length=True,
#                           truncation=True)
# #     print(tmp_dict['input_ids'].size())
# #     print(tmp_dict['attention_mask'].size())
# #     print(tmp_dict['token_type_ids'].size())

#     if i == 0 :
#         input_dict = tmp_dict
#     else:
        
#         input_dict['input_ids'] = torch.cat((input_dict['input_ids'],tmp_dict['input_ids']),dim=0)
#         input_dict['attention_mask'] = torch.cat((input_dict['attention_mask'],tmp_dict['attention_mask']),dim=0)
#         input_dict['token_type_ids'] = torch.cat((input_dict['token_type_ids'],tmp_dict['token_type_ids']),dim=0)
        
    

# print(input_dict['input_ids'].size())
# print(input_dict['attention_mask'].size())
# print(input_dict['token_type_ids'].size())


# # with open('input_dict{}'.format(len(x_qd)), 'wb') as handle:
# #     pickle.dump(input_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# # with open('train_y{}'.format(len(x_qd)), 'wb') as handle:
# #     pickle.dump(train_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/124168 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 124168/124168 [3:41:08<00:00,  9.36it/s]   

torch.Size([124168, 512])
torch.Size([124168, 512])
torch.Size([124168, 512])


In [16]:
pretrained_model = 'roberta-large'
with open('{}_input_dict{}'.format(pretrained_model,len(x_qd)), 'wb') as handle:
    pickle.dump(input_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('{}_train_y{}'.format(pretrained_model,len(x_qd)), 'wb') as handle:
    pickle.dump(train_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
torch.save(input_dict['input_ids'],'input_ids{}'.format(len(x_qd)))
torch.save(input_dict['attention_mask'],'attention_mask{}'.format(len(x_qd)))
torch.save(input_dict['token_type_ids'],'token_type_ids{}'.format(len(x_qd)))


In [16]:
with open('input_dict', 'rb') as handle:
    input_dict = pickle.load(handle)

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TrainDataset(Dataset):
    def __init__(self, input_dict, y):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y
        
    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return inputid , tokentype , attentionmask, y
    
    def __len__(self):
        return len(self.input_ids)
    
# def get_predictions(model, loader, BATCH_SIZE):
#     result = []
#     total_count = 0 # 第n筆data
#     model.eval()
#     with torch.no_grad():
#         for data in loader:
#             tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]

#             tokens_tensors, segments_tensors, masks_tensors = data[:3]
#             outputs = model(input_ids=tokens_tensors, 
#                       token_type_ids=segments_tensors, 
#                       attention_mask=masks_tensors)
#             pred = outputs[0].argmax(1)

#             result.append(r)
#             total_count += 1

#     return result

In [ ]:
x_qd = []
train_y = []

for query_idx,query in enumerate(train_queries):
    query = ' '.join(query)
#     BM25找回來的
    bm25_topk = BM25[query_idx]
#     query的正確答案
    query_pos_doc_idx = []
    query_pos_doc_id = train_queries_pos_docs[query_idx].split()
    count = 0
    for doc_id in query_pos_doc_id:
        doc_idx = doc_id2idx[doc_id]
        if doc_idx in bm25_topk:
            count += 1
        else:
            document = ' '.join(docs[doc_idx])
            x_qd.append((query,document))
            train_y.append(1)
        query_pos_doc_idx.append(doc_idx)
#     print('label num : {} , BM25 find num {}'.format(len(query_pos_doc_id),count))
    count = 0
    for doc_idx in bm25_topk:
        document = ' '.join(docs[doc_idx])
        if doc_idx in query_pos_doc_idx:
            x_qd.append((query,document))
            train_y.append(1)
            count += 1
        else:
#             if np.random.random()<0.1:
            x_qd.append((query,document))
            train_y.append(0)
            
#     print('in {} data , {} data label is true, {} data label is false '.format(len(bm25_topk),count,len(bm25_topk)-count))


# x_d = np.array(x_d)
# x_q = np.array(x_q)
# y = np.array(y)
print(len(x_qd))
print(len(train_y))

print(sum(train_y) / len(train_y))
import torch
train_y = torch.tensor(train_y)

    

In [19]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)
weight = torch.FloatTensor([1, 10]).cuda()
binary_loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
BATCH_SIZE = 4
trainSet = TrainDataset(input_dict, y)
trainLoader = DataLoader(trainSet, batch_size=BATCH_SIZE,shuffle=True)
model = RobertaForSequenceClassification.from_pretrained(pretrained_model)

device = 'cuda:0'
model = model.to(device)
model.train() 

EPOCHS = 4

for epoch in range(EPOCHS):
    running_loss = 0.0
    count = 0
    correct_count = 0
    for data in trainLoader:
        tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
        optimizer.zero_grad()
        outputs = model(input_ids=tokens_tensors, 
                      token_type_ids=segments_tensors, 
                      attention_mask=masks_tensors)
        
        pred = outputs[0].argmax(1)
        count += labels.size()[0]
        correct_count += (labels==pred).sum().cpu().item()
        
        loss = binary_loss_fct(outputs[0],labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        print(f'\r epoch:{epoch}/{EPOCHS} acc rate : {correct_count/count} data:{count}/{len(trainLoader)}',end='')
        
    CHECKPOINT_NAME = './roberta_large_epoch{}_train_size{}'.format(epoch,20872) 
    torch.save(model.state_dict(), CHECKPOINT_NAME)




 epoch:0/4 acc rate : 0.4350594227504245 data:2356/52188

IndexError: index 20861 is out of bounds for dimension 0 with size 20847